In [1]:
import pandas as pd
import polars as pl

import plotly.express as px

from collections import Counter


pd.set_option('display.max_columns', None)

## Introduction et contexte

Ce notebook a pour but de faire une première exploration des données mises à disposition par openfoodfacts pour répondre à notre problème qui est:

quelle alternative à un produit alimentaire pourrait nous être proposé ?

Dans ces alternatives, quels sont les produits qui ne contiennent pas de produit susceptible de provoquer une allergie alimentaire ?

Le volume de données est important, nous allons nous concentrer sur les produits où la liste des ingrédients est renseignés et ou les produits sont vendus en france.

## Chargement des Données

In [ ]:
rawdata = None
def extract_raw_data():
    file_path = '../data/en.openfoodfacts.org.products.csv.gz'
    chunk_size = 10000

    filtered_chunks_list = []

    for chunk in pd.read_csv(file_path, chunksize=chunk_size, compression='gzip', sep='\t', engine='python', quoting=3):
        filtered_chunks = chunk[(chunk['countries_tags'] == 'en:france') & (chunk['ingredients_tags'].notna())]
        filtered_chunks_list.append(filtered_chunks)

    if filtered_chunks_list:
        rawdata = pd.concat(filtered_chunks_list, axis=0)
    return rawdata

# rawdata = extract_raw_data()
rawdata = pd.read_csv('filtered_dataset_openfoodfacts_raw_en.csv')
display(rawdata)

/tmp/ipykernel_19014/1538594827.py:17: DtypeWarning: Columns (12,18,32,33,34,36,54,73) have mixed types. Specify dtype option on import or set low_memory=False.
  rawdata = pd.read_csv('filtered_dataset_openfoodfacts_raw_en.csv')


,Unnamed: 0,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,ecoscore_score,ecoscore_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
0,0,1,http://world-en.openfoodfacts.org/product/0000...,inf,1634745456,2021-10-20T15:57:36Z,1730771746,2024-11-05T01:55:46Z,smoothie-app,1.730772e+09,2024-11-05T01:55:46Z,Purée Mix Tropical Harmony + Aloe,NaN,NaN,1 kg,NaN,NaN,NaN,NaN,Punuts,punuts,syrups,en:syrups,Syrups,Spanien,en:spain,Spain,NaN,NaN,"No gluten, Vegetarian, No artificial flavors, ...","en:no-gluten,en:vegetarian,en:no-artificial-fl...","No gluten,Vegetarian,No artificial flavors,Veg...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Vereinigte Staaten von Amerika, Germany","en:germany,en:united-states","Germany,United States","fruit juice blend, guava and papaya purees, pa...","en:fr

## Inspection des Données Brutes

In [3]:
# Basic stats
print("Taille du dataset:")
print("Number of rows : {}".format(rawdata.shape[0]))
print("Number of columns : {}".format(rawdata.shape[1]))
print()
print("---------------------------")
print()

print("Basics infos:")
print()
display(rawdata.info())
print()
print("---------------------------")
print()

print("Basics statistics: ")
print()
data_desc = rawdata.describe(include='all')
display(data_desc)
print()
print("---------------------------")
print()

print("Unique elements by feature: ")
print()
display(rawdata.nunique().sort_values())
print()
print("---------------------------")
print()

print("Percentage of missing values: ")
print()
null_percentage = 100 * rawdata.isnull().mean()
null_percentage_df = null_percentage.to_frame(name='Null Percentage').T
display(null_percentage_df) 

Taille du dataset:
Number of rows : 567867
Number of columns : 207

---------------------------

Basics infos:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567867 entries, 0 to 567866
Columns: 207 entries, Unnamed: 0 to acidity_100g
dtypes: float64(131), int64(4), object(72)
memory usage: 896.8+ MB


None


---------------------------

Basics statistics: 



,Unnamed: 0,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,ecoscore_score,ecoscore_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
count,567867.000000,5.678670e+05,567867,567867,5.678670e+05,567867,5.678670e+05,567867,556437,5.678650e+05,567865,550590,30,7029,86369,19712,19714,19712,926,301849,301824,281911,281911,281911,6828,6824,6822,7176,7173,104024,104030,104027,7238,7240,4476,0.0,5040,12399,35860,565147,565147,565147,258004,257917,262799,61530,0.0,5886,8544,8494,283372,2.810160e+05,3515,258005.000000,2,170433,170433,244767.000000,566007,243393.000000,566034,566034,219415,219415,219415,567867,567867,567867,196037,103508.000000,566006,256349,8.372700e+04,2657,24446,66221.000000,65859,567867.000000,3.364590e+05,336459,281911,281911,318197,318197,76896,76896,22


---------------------------

Unique elements by feature: 



erucic-acid_100g                         0
cities                                   0
elaidic-acid_100g                        0
dihomo-gamma-linolenic-acid_100g         0
unsaturated-fat_100g                     0
                                     ...  
created_t                           418300
created_datetime                    418300
code                                567834
url                                 567835
Unnamed: 0                          567867
Length: 207, dtype: int64


---------------------------

Percentage of missing values: 



,Unnamed: 0,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,ecoscore_score,ecoscore_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
Null Percentage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.012795,0.000352,0.000352,3.042438,99.994717,98.76221,84.790629,96.528765,96.528412,96.528765,99.836934,46.845124,46.849526,50.356157,50.356157,50.356157,98.797606,98.79831,98.798662,98.736324,98.736852,81.681626,81.68057,81.681098,98.725406,98.725054,99.211787,100.0,99.112468,97.816566,93.685141,0.478985,0.478985,0.478985,54.566122,54.581442,53.721734,89.164716,100.0,98.96349,98.495422,98.504227,50.098879,50.513765,99.381017,54.565946,99.999648,69.987162,69.987162,56.897126,0.327541,57.139084,0.322787,0.322787,61.361551,61.361551,61.361551,0.0,0.0,0.0,65.47836,81.772493,0.327718,54.85

**Notes :**

Il y a beaucoup de données manquantes.

Nous allons dans un premier temps nous concentrer sur les données des produits vendus en France.

## Analyse Exploratoire des Données

In [4]:
# Find different states
unique_values = set()
rawdata['states_tags'].str.split(',').apply(unique_values.update)

states_df = pd.DataFrame(data=unique_values, columns=['states'])
display(states_df)

,states
0,en:photos-uploaded
1,en:nutrition-facts-completed
2,en:front-photo-to-be-selected
3,en:to-be-checked
4,en:packaging-code-completed
5,en:packaging-code-to-be-completed
6,en:origins-to-be-completed
7,en:nutrition-facts-to-be-completed
8,en:product-name-completed
9,en:categories-to-be-completed


In [5]:
# Vérifier que la colonne 'state' existe
if 'state' in rawdata.columns:
    # Compter les occurrences de chaque état
    states_count = rawdata['state'].value_counts().reset_index()
    states_count.columns = ['state', 'count']
    
    # Afficher le tableau
    print(states_count)
else:
    print("La colonne 'state' n'existe pas dans le dataset.")

La colonne 'state' n'existe pas dans le dataset.


**Notes :**

Il y a beaucoup de colonnes, nous allons sélectionner celles qui potentiellement pourront aider nos futurs modèles à apprendre et à répondre à notre problématique.

In [ ]:
columns_to_keep = [
    'code',
    'url',
    'created_datetime',
    'last_modified_datetime',
    'product_name',
    'packaging',
    'categories_tags',
    'ingredients_tags',
    'ingredients_analysis_tags',
    'allergens',
    'traces_tags',
    'additives_n',
    'additives_tags',
    'nutriscore_grade',
    'food_groups_tags',
    'states_tags',
    'ecoscore_score',
    'ecoscore_grade',
    'nutrient_levels_tags',
    'popularity_tags',
    'main_category',
    'image_url',
    'image_small_url',
    'image_ingredients_url',
    'image_ingredients_small_url',
    'image_nutrition_url',
    'image_nutrition_small_url',
    'energy-kcal_100g',
    'energy_100g',
    'fat_100g',
    'cholesterol_100g',
    'sugars_100g',
    'proteins_100g',
    'salt_100g',
    'alcohol_100g'
]

data = rawdata[columns_to_keep]
display(data)

,code,url,created_datetime,last_modified_datetime,product_name,packaging,categories_tags,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_score,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,alcohol_100g
0,1,http://world-en.openfoodfacts.org/product/0000...,2021-10-20T15:57:36Z,2024-11-05T01:55:46Z,Purée Mix Tropical Harmony + Aloe,NaN,en:syrups,"fruit juice blend, guava and papaya purees, pa...","en:fruit-juice-blend,en:guava,en:fruit,en:papa...","en:may-contain-palm-oil,en:vegan,en:vegetarian",NaN,NaN,4.0,"en:e160a,en:e160ai,en:e330,en:e415,en:e440",-5.0,a,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-moderate-quantity,en:saturated-fat-i...",NaN,en:syrups,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,360.000000,1530.0,10.000000,0.000,8.000000,49.000000,0.020000,NaN
1,2,http://world-en.openfoodfacts.org/product/0000...,2024-08-02T13:47:35Z,2024-10-14T14:03:32Z,Poudre de grillon,NaN,"en:beverages-and-beverages-preparations,en:bev...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-moderate-quantity,en:saturated-fat-i...",NaN,en:instant-beverages,NaN,NaN,NaN,NaN,NaN,NaN,400.000000,1674.0,16.444445,NaN,4.888889,10.000000,0.013123,NaN
2,3,http://world-en.openfoodfacts.org/product/0000...,2024-05-27T13:59:03Z,2024-10-25T13:23:44Z,Feuchtes Toilettentuch - Kamille,NaN,NaN,Kein Rock gleicht den anderen Wir haben nachge...,en:kein-rock-gleicht-den-anderen-wir-haben-nac...,"en:palm-oil-content-unknown,en:vegan-status-un...",NaN,NaN,0.0,NaN,NaN,unknown,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,NaN,NaN,NaN,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,160.000000,669.0,3.200000,0.002,1.400000,8.700000,0.807087,NaN
3,4,http://world-en.openfoodfacts.org/product/0000...,2019-06-10T14:20:26Z,2024-10-30T19:35:25Z,Marinara,NaN,"en:snacks,en:sweet-snacks,en:cocoa-and-its-pro...","Organic cocoa mass 60%, organic cane sugar, or...","en:cocoa-paste,en:plant,en:cocoa,en:cane-sugar...","en:palm-oil-free,en:vegan,en:vegetarian",NaN,NaN,0.0,NaN,23.0,e,"en:sugary-snacks,en:chocolate-products","en:to-be-completed,en:nutrition-facts-complete...",41.0,d,"en:fat-in-low-quantity,en:saturated-fat-in-hig...",NaN,en:dark-chocolate-bar,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,56.451613,2401.0,2.016129,NaN,35.000000,1.612903,0.000000,NaN
4,5,http://world-en.openfoodfacts.org/product/0000...,2020-11-14T07:08:40Z,2024-10-20T14:02:34Z,Bio inulin,Glass,"en:plant-based-foods-and-beverages,en:plant-ba...","Jus et purée d'abricots (50%), eau, sucre.","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",NaN,NaN,0.0,NaN,-2.0,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:vegetables,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.o

In [7]:
# Liste des états cibles
target_states = ["en:completed", "en:origins-completed", "en:ingredients-completed"]

# Construire une expression régulière pour les états
pattern = '|'.join(target_states)

# Compter les lignes contenant au moins un des états cibles
count = data['states_tags'].str.contains(pattern).sum()

print("Nombre de lignes avec les états spécifiés:", count)


Nombre de lignes avec les états spécifiés: 258229


In [8]:
data = data[data['states_tags'].str.contains(pattern)]

In [9]:
count_data_allergen = data['allergens'].notna().sum()
print("Nombre de lignes avec des allergens:", count_data_allergen)

Nombre de lignes avec des allergens: 61186


**Notes:**

Nous allons remplacer les valeurs de la colonne allergens non renseignées par la valeur `en:none`.

In [10]:
data.loc[:, 'allergens'] = data['allergens'].astype(str)
data['allergens'] = data['allergens'].str.lower()
data['allergens'] = data['allergens'].apply(lambda x: 'en:none' if x == 'nan' else x)
data['allergens'].value_counts()

allergens
en:none                                                                       197056
en:milk                                                                        15769
en:gluten                                                                       9401
en:gluten,en:milk                                                               4692
en:nuts                                                                         2931
                                                                               ...  
en:milk,en:palm oil                                                                1
en:crustaceans,en:gluten,en:milk,en:molluscs,en:soybeans                           1
en:celery,en:eggs,en:gluten,en:milk,en:sesame-seeds                                1
en:celery,en:crustaceans,en:gluten,en:milk,en:sesame-seeds                         1
en:eggs,en:gluten,en:milk,en:sesame-seeds,en:sulphur-dioxide-and-sulphites         1
Name: count, Length: 816, dtype: int64

In [11]:
# Divise chaque entrée par ',' et compile tous les allergènes en une seule liste
allergens_list = data["allergens"].fillna('').str.split(',').sum()

# Compte les occurrences de chaque allergène
allergen_counts = Counter(allergens_list)

# Convertir le résultat en DataFrame
allergens_df = pd.DataFrame(allergen_counts.items(), columns=['allergen', 'count']).sort_values(by='count', ascending=False)
display(allergens_df)

,allergen,count
0,en:none,197061
3,en:milk,31839
2,en:gluten,24703
4,en:soybeans,11187
1,en:eggs,6832
...,...,...
156,en:nonfat milk,1
157,en:egg. sulfate,1
158,en:thiamine,1
159,fr:wheat gluten,1


**Notes :**

On peu se fier seulement aux allergènes contenant plus de 100 produits dans la base de données.

Nous pourronts mettre les autres dans une autre catégories nommée `other`.

In [12]:
display(allergens_df[allergens_df['count'] > 100])

,allergen,count
0,en:none,197061
3,en:milk,31839
2,en:gluten,24703
4,en:soybeans,11187
1,en:eggs,6832
6,en:nuts,6562
9,en:peanuts,4225
5,en:celery,2849
10,en:sesame-seeds,1698
13,en:sulphur-dioxide-and-sulphites,1435


**Notes:**

Nous allons considérer que la valeur vide correspond à pas d'allergène : `en:none`.

Nous allons mettre les allergènes qui ne sont pas dans cette liste dans une catégorie `other`.

Nous pouvons fusionner les valeurs `fr:Non` et `en:none` et traduire `fr:avoine` en `en:oats`.

In [13]:
# Liste des allergènes à conserver (ceux dont le count est supérieur à 100)
allergens_to_keep = allergens_df[allergens_df['count'] > 100]['allergen'].tolist()
allergens_to_keep.append('en:none') # Ajoute l'entrée none

data['allergens'] = data['allergens'].apply(lambda x: 'other' if x not in allergens_to_keep  else x)
data['allergens'] = data['allergens'].apply(lambda x: 'en:oats' if x == 'fr:avoine' else x)
data['allergens'].value_counts()

display(data)

,code,url,created_datetime,last_modified_datetime,product_name,packaging,categories_tags,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_score,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,alcohol_100g
0,1,http://world-en.openfoodfacts.org/product/0000...,2021-10-20T15:57:36Z,2024-11-05T01:55:46Z,Purée Mix Tropical Harmony + Aloe,NaN,en:syrups,"fruit juice blend, guava and papaya purees, pa...","en:fruit-juice-blend,en:guava,en:fruit,en:papa...","en:may-contain-palm-oil,en:vegan,en:vegetarian",en:none,NaN,4.0,"en:e160a,en:e160ai,en:e330,en:e415,en:e440",-5.0,a,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-moderate-quantity,en:saturated-fat-i...",NaN,en:syrups,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,360.000000,1530.0,10.000000,0.000,8.000000,49.000000,0.020000,NaN
1,2,http://world-en.openfoodfacts.org/product/0000...,2024-08-02T13:47:35Z,2024-10-14T14:03:32Z,Poudre de grillon,NaN,"en:beverages-and-beverages-preparations,en:bev...",NaN,NaN,NaN,en:none,NaN,NaN,NaN,NaN,unknown,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-moderate-quantity,en:saturated-fat-i...",NaN,en:instant-beverages,NaN,NaN,NaN,NaN,NaN,NaN,400.000000,1674.0,16.444445,NaN,4.888889,10.000000,0.013123,NaN
2,3,http://world-en.openfoodfacts.org/product/0000...,2024-05-27T13:59:03Z,2024-10-25T13:23:44Z,Feuchtes Toilettentuch - Kamille,NaN,NaN,Kein Rock gleicht den anderen Wir haben nachge...,en:kein-rock-gleicht-den-anderen-wir-haben-nac...,"en:palm-oil-content-unknown,en:vegan-status-un...",en:none,NaN,0.0,NaN,NaN,unknown,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,NaN,NaN,NaN,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,160.000000,669.0,3.200000,0.002,1.400000,8.700000,0.807087,NaN
3,4,http://world-en.openfoodfacts.org/product/0000...,2019-06-10T14:20:26Z,2024-10-30T19:35:25Z,Marinara,NaN,"en:snacks,en:sweet-snacks,en:cocoa-and-its-pro...","Organic cocoa mass 60%, organic cane sugar, or...","en:cocoa-paste,en:plant,en:cocoa,en:cane-sugar...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,NaN,0.0,NaN,23.0,e,"en:sugary-snacks,en:chocolate-products","en:to-be-completed,en:nutrition-facts-complete...",41.0,d,"en:fat-in-low-quantity,en:saturated-fat-in-hig...",NaN,en:dark-chocolate-bar,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,56.451613,2401.0,2.016129,NaN,35.000000,1.612903,0.000000,NaN
4,5,http://world-en.openfoodfacts.org/product/0000...,2020-11-14T07:08:40Z,2024-10-20T14:02:34Z,Bio inulin,Glass,"en:plant-based-foods-and-beverages,en:plant-ba...","Jus et purée d'abricots (50%), eau, sucre.","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,NaN,0.0,NaN,-2.0,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:vegetables,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://im

In [14]:
# Séparer les allergènes par ",", et convertir en une liste
allergens_series = data['allergens'].str.split(',').explode().str.strip()
# Compter les occurrences de chaque allergène
allergens_counts = allergens_series.value_counts().reset_index()
allergens_counts.columns = ['allergen', 'count']

# Créer le graphique
fig = px.bar(allergens_counts, x='allergen', y='count', 
             title="Répartition des allergènes dans le dataset",
             labels={'allergen': 'Allergènes', 'count': 'Nombre d\'occurrences'},
             height=500)

fig.show()

Maintenant nous allons nous focaliser sur les traces d'allergènes.

In [15]:
data.loc[:, 'traces_tags'] = data['traces_tags'].astype(str).str.lower()

# Divise chaque entrée par ',' et compile tous les traces d'allergènes en une seule liste
traces_allergens_list = data["traces_tags"].fillna('').str.split(',').sum()

# Compte les occurrences de chaque traces d'allergène
traces_allergen_counts = Counter(traces_allergens_list)

# Convertir le résultat en DataFrame
traces_allergens_df = pd.DataFrame(traces_allergen_counts.items(), columns=['allergen', 'count']).sort_values(by='count', ascending=False)
display(allergens_df)

,allergen,count
0,en:none,197061
3,en:milk,31839
2,en:gluten,24703
4,en:soybeans,11187
1,en:eggs,6832
...,...,...
156,en:nonfat milk,1
157,en:egg. sulfate,1
158,en:thiamine,1
159,fr:wheat gluten,1


In [16]:
display(traces_allergens_df[traces_allergens_df['count'] > 20])

,allergen,count
0,nan,249861
1,en:nuts,3669
2,en:soybeans,2993
3,en:milk,2664
6,en:gluten,2213
9,en:peanuts,1884
16,en:eggs,1785
10,en:sesame-seeds,1022
8,en:mustard,449
38,en:sulphur-dioxide-and-sulphites,367


In [17]:
# Séparer les allergènes par ",", et convertir en une liste
data.loc[:, 'traces_tags'] = data['traces_tags'].astype(str)

traces_allergens_to_keep = traces_allergens_df[traces_allergens_df['count'] > 20]['allergen'].tolist()

data['traces_tags'] = data['traces_tags'].apply(lambda x: 'en:none' if (x == 'nan')  else x)
data['traces_tags'] = data['traces_tags'].apply(lambda x: 'other' if x not in traces_allergens_to_keep  else x)

traces_allergens_series = data['traces_tags'].str.split(',').explode().str.strip()
traces_allergens_counts = traces_allergens_series.value_counts().reset_index()
traces_allergens_counts.columns = ['traces_tags', 'count']

# Créer le graphique
fig = px.bar(traces_allergens_counts, x='traces_tags', y='count', 
             title="Répartition des traces d'allergènes dans le dataset",
             labels={'traces_tags': 'Traces allergènes', 'count': 'Nombre d\'occurrences'},
             height=500)

fig.show()

### Sélections et nettoyage des données

In [19]:
data.describe(include='all')

,code,url,created_datetime,last_modified_datetime,product_name,packaging,categories_tags,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_score,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,alcohol_100g
count,2.582290e+05,258229,258229,258229,255822,13965,225564,257208,257122,257147,258229,258229,257209.000000,169892,209047.000000,258218,173654,258229,66229.000000,258216,215402,39090,225564,76981,76981,47127,47127,55942,55942,243743.000000,244633.000000,244964.000000,183157.000000,235393.000000,244657.000000,2.417430e+05,1504.000000
unique,NaN,258207,116513,151277,177247,3860,12364,193195,173732,36,15,18,NaN,43298,NaN,7,44,1613,NaN,9,222,11533,6695,76962,76962,47107,47107,55932,55932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,http://world-en.openfoodfacts.org/product/0202...,2020-04-23T18:23:21Z,2022-08-01T12:48:51Z,Ice cream,Plastic,en:snacks,"Carbonated water, natural flavor.","en:carbonated-water,en:water,en:natural-flavou...","en:palm-oil-free,en:vegan-status-unknown,en:ve...",en:none,en:none,NaN,en:e330,NaN,d,"en:sugary-snacks,en:sweets","en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...","top-75-percent-scans-2023,top-80-percent-scans...",en:groceries,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,2,27,90,495,826,18931,431,775,48253,197056,249911,NaN,11811,NaN,63867,18459,82978,NaN,184349,17113,6006,20470,2,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,5.260223e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.686092,NaN,8.627342,NaN,NaN,NaN,49.762294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263.769653,1102.914212,12.315812,0.032443,15.574527,6.998107,3.309300e+08,1.195133
std,3.887996e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.390585,NaN,8.990085,NaN,NaN,NaN,20.844618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210.901112,880.613925,206.133901,0.894173,21.243031,13.766187,1.627095e+11,5.633088
min,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,-15.000000,NaN,NaN,NaN,-30.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,2.800036e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,1.000000,NaN,NaN,NaN,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.000000,368.000000,0.000000,0.000000,1.180000,0.010000,8.500000e-02,0.000000
50%,4.240022e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,10.000000,NaN,NaN,NaN,47.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,261.000000,1092.000000,4.410000,0.000000,5.360000,4.350000,6.750000e-01,0.000000
75%,7.329628e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,15.000000,NaN,NaN,NaN,68.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,393.000000,1644.000000,18.580000,0.022000,22.860000,10.000000,1.482500e+00,0.000000


In [23]:
data[data['additives_n'] == 1.0].head()

,code,url,created_datetime,last_modified_datetime,product_name,packaging,categories_tags,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_score,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,alcohol_100g
49,51,http://world-en.openfoodfacts.org/product/0000...,2016-12-01T19:59:24Z,2024-10-11T08:26:38Z,Fondants Citron,"Boîte carton, Sachet Plastique","en:plant-based-foods-and-beverages,en:plant-ba...","mûre bio 50% (fruit 27%, purée 23%) sucre de c...","en:blackberry,en:fruit,en:berries,en:cane-suga...","en:palm-oil-free,en:vegan-status-unknown,en:ve...",other,other,1.0,en:e440,10.0,c,"en:sugary-snacks,en:sweets","en:to-be-completed,en:nutrition-facts-complete...",64.0,b,"en:fat-in-low-quantity,en:saturated-fat-in-low...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:jams,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,249.00,1056.0,0.30,NaN,59.00,0.5,0.050,NaN
50,52,http://world-en.openfoodfacts.org/product/0000...,2023-09-06T09:13:41Z,2024-10-11T22:17:20Z,Organic Erythritol Granulated,NaN,fr:croquettes-chat,100% certified organic granulated erythritol,en:e968,"en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,1.0,en:e968,0.0,b,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...","top-75-percent-scans-2023,top-80-percent-scans...",fr:croquettes-chat,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,0.00,0.0,0.00,NaN,0.00,0.0,0.000,NaN
69,75,http://world-en.openfoodfacts.org/product/0000...,2020-04-08T13:01:34Z,2024-10-04T09:59:29Z,NaN,NaN,NaN,"potato, high oleic sunflower oil (31%) flavour...","en:potato,en:vegetable,en:root-vegetable,en:tu...","en:palm-oil-free,en:vegan-status-unknown,en:ve...",en:none,en:none,1.0,en:e330,NaN,unknown,NaN,"en:to-be-completed,en:nutrition-facts-to-be-co...",NaN,unknown,NaN,NaN,NaN,NaN,NaN,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,83,http://world-en.openfoodfacts.org/product/0000...,2022-06-06T14:47:37Z,2024-10-04T09:41:25Z,Xanthan gum,NaN,"en:cooking-helpers,en:xanthan-gum",xanthan gum,en:e415,"en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,1.0,en:e415,-6.0,a,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:xanthan-gum,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,183.00,766.0,0.00,NaN,0.10,5.8,0.050,NaN
102,112,http://world-en.openfoodfacts.org/product/0000...,2021-05-10T11:26:28Z,2024-10-04T09:43:04Z,Plus,NaN,NaN,wate NUTRITIONAL INFORMATION for 250 ml of pro...,it:wate-nutritional-information-for-250-ml-of-...,"en:may-contain-palm-oil,en:vegan,en:vegetarian",en:none,en:none,1.0,en:e101,NaN,unknown,NaN,"en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,NaN,NaN,NaN,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,215.13,900.0,6.79,NaN,19.77,

- url --> ok
- created_datetime --> transform to 1 column for year
- last_modified_datetime --> transform to 1 column for year
- product_name --> delete empty rows
- packaging --> set 'unknown' for empty rows
- categories_tags --> set 'en:unknown' for empty rows
- ingredients_tags --> delete empty rows
- ingredients_analysis_tags --> set 'unknown' for empty rows
- allergens --> set 'en:none' for empty rows
- traces_tags --> set 'en:none' for empty rows
- additives_n --> ok
- additives_tags --> set en:none when empty
- nutriscore_grade --> set unknown when empty
- food_groups_tags --> set en:none when empty
- states_tags --> ok
- ecoscore_grade --> set unknown when empty
- nutrient_levels_tags --> set en:unknown when empty
- popularity_tags --> set unknown when empty
- main_category --> set en:none when empty
- image_url --> set 'en:none' for empty rows
- energy-kcal_100g --> set -1 for empty rows
- energy_100g --> set -1 for empty rows
- fat_100g --> set -1 for empty rows
- cholesterol_100g --> set -1 for empty rows
- sugars_100g --> set -1 for empty rows
- proteins_100g --> set -1 for empty rows
- salt_100g --> set -1 for empty rows


In [ ]:
columns_to_keep = [
    'code',
    'url',
    'last_modified_datetime',
    'product_name',
    'packaging',
    'categories_tags',
    'ingredients_tags',
    'ingredients_analysis_tags',
    'allergens',
    'traces_tags',
    'additives_n',
    'additives_tags',
    'nutriscore_grade',
    'food_groups_tags',
    'states_tags',
    'ecoscore_grade',
    'nutrient_levels_tags',
    'popularity_tags',
    'main_category',
    'image_url',
    'energy_100g',
    'fat_100g',
    'cholesterol_100g',
    'sugars_100g',
    'proteins_100g',
    'salt_100g',
]

clean_data = data[columns_to_keep]
display(clean_data)

,code,url,created_datetime,last_modified_datetime,product_name,packaging,categories_tags,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_grade,food_groups_tags,states_tags,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g
0,1,http://world-en.openfoodfacts.org/product/0000...,2021-10-20T15:57:36Z,2024-11-05T01:55:46Z,Purée Mix Tropical Harmony + Aloe,NaN,en:syrups,"en:fruit-juice-blend,en:guava,en:fruit,en:papa...","en:may-contain-palm-oil,en:vegan,en:vegetarian",en:none,en:none,4.0,"en:e160a,en:e160ai,en:e330,en:e415,en:e440",a,NaN,"en:to-be-completed,en:nutrition-facts-complete...",unknown,"en:fat-in-moderate-quantity,en:saturated-fat-i...",NaN,en:syrups,https://images.openfoodfacts.org/images/produc...,360.000000,1530.0,10.000000,0.000,8.000000,49.000000,0.020000
1,2,http://world-en.openfoodfacts.org/product/0000...,2024-08-02T13:47:35Z,2024-10-14T14:03:32Z,Poudre de grillon,NaN,"en:beverages-and-beverages-preparations,en:bev...",NaN,NaN,en:none,en:none,NaN,NaN,unknown,NaN,"en:to-be-completed,en:nutrition-facts-complete...",unknown,"en:fat-in-moderate-quantity,en:saturated-fat-i...",NaN,en:instant-beverages,NaN,400.000000,1674.0,16.444445,NaN,4.888889,10.000000,0.013123
2,3,http://world-en.openfoodfacts.org/product/0000...,2024-05-27T13:59:03Z,2024-10-25T13:23:44Z,Feuchtes Toilettentuch - Kamille,NaN,NaN,en:kein-rock-gleicht-den-anderen-wir-haben-nac...,"en:palm-oil-content-unknown,en:vegan-status-un...",en:none,en:none,0.0,NaN,unknown,NaN,"en:to-be-completed,en:nutrition-facts-complete...",unknown,NaN,NaN,NaN,https://images.openfoodfacts.org/images/produc...,160.000000,669.0,3.200000,0.002,1.400000,8.700000,0.807087
3,4,http://world-en.openfoodfacts.org/product/0000...,2019-06-10T14:20:26Z,2024-10-30T19:35:25Z,Marinara,NaN,"en:snacks,en:sweet-snacks,en:cocoa-and-its-pro...","en:cocoa-paste,en:plant,en:cocoa,en:cane-sugar...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,0.0,NaN,e,"en:sugary-snacks,en:chocolate-products","en:to-be-completed,en:nutrition-facts-complete...",d,"en:fat-in-low-quantity,en:saturated-fat-in-hig...",NaN,en:dark-chocolate-bar,https://images.openfoodfacts.org/images/produc...,56.451613,2401.0,2.016129,NaN,35.000000,1.612903,0.000000
4,5,http://world-en.openfoodfacts.org/product/0000...,2020-11-14T07:08:40Z,2024-10-20T14:02:34Z,Bio inulin,Glass,"en:plant-based-foods-and-beverages,en:plant-ba...","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,0.0,NaN,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:vegetables,https://images.openfoodfacts.org/images/produc...,208.000000,840.0,0.000000,NaN,8.000000,0.000000,0.230000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567796,2661427,http://world-en.openfoodfacts.org/product/0266...,2017-03-09T14:40:34Z,2022-06-21T16:28:13Z,Sunflower Seeds,NaN,"en:plant-based-foods-and-beverages,en:plant-ba...","en:sunflower-seed,en:plant,en:seed,en:sunflowe...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,0.0,NaN,unknown,NaN,"en:to-be-completed,en:nutrition-facts-complete...",d,"en:fat-in-high-quantity,en:saturated-fat-in-mo...","bottom-25-percent-scans-2019,bottom-20-percent...",en:sunflower-seeds,https://images.openfoodfacts.org/images/produc...,615.000000,2573.0,53.850000,0.000,0.000000,30.770000,NaN
567806,266147009836,http://world-en.openfoodfacts.org/product/0266...,2024-07-06T12:25:38Z,2024-07-06T12:27:16Z,Roast beef,NaN,NaN,"en:rewards-savings-3-28-gredients,en:water,en:...","en:palm-oil-content-unknown,en:non-vegan,en:no...",en:none,en:none,0.0,NaN,unknown,NaN,"en:to-be-completed,en:nutrition-facts-to-be-co...",unknown,NaN,NaN,NaN,NaN,

In [ ]:
import pandas as pd

def clean_dataset(df):
    # Convert 'created_datetime' et 'last_modified_datetime' en années
    df['created_year'] = pd.to_datetime(df['created_datetime'], errors='coerce').dt.year
    df['last_modified_year'] = pd.to_datetime(df['last_modified_datetime'], errors='coerce').dt.year
    df.drop(['last_modified_datetime'], axis=1, inplace=True)

    # Supprimer les lignes où 'product_name' ou 'ingredients_tags' sont vides
    df.dropna(subset=['product_name', 'ingredients_tags'], inplace=True)

    # Remplacer les valeurs manquantes par des valeurs par défaut
    df['packaging'].fillna('unknown', inplace=True)
    df['categories_tags'].fillna('en:unknown', inplace=True)
    df['ingredients_analysis_tags'].fillna('unknown', inplace=True)
    df['allergens'].fillna('en:none', inplace=True)
    df['traces_tags'].fillna('en:none', inplace=True)
    df['additives_tags'].fillna('en:none', inplace=True)
    df['nutriscore_grade'].fillna('unknown', inplace=True)
    df['food_groups_tags'].fillna('en:none', inplace=True)
    df['ecoscore_grade'].fillna('unknown', inplace=True)
    df['nutrient_levels_tags'].fillna('en:unknown', inplace=True)
    df['popularity_tags'].fillna('unknown', inplace=True)
    df['main_category'].fillna('en:none', inplace=True)
    df['image_url'].fillna('en:none', inplace=True)

    return df

clean_data = clean_dataset(clean_data)
clean_data.head()


/tmp/ipykernel_19014/2703447646.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_19014/2703447646.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_19014/2703447646.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_19014/2703447646.py:10: SettingWithCopyWarning:


A value 

,code,url,product_name,packaging,categories_tags,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_grade,food_groups_tags,states_tags,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,created_year,last_modified_year
0,1,http://world-en.openfoodfacts.org/product/0000...,Purée Mix Tropical Harmony + Aloe,unknown,en:syrups,"en:fruit-juice-blend,en:guava,en:fruit,en:papa...","en:may-contain-palm-oil,en:vegan,en:vegetarian",en:none,en:none,4.0,"en:e160a,en:e160ai,en:e330,en:e415,en:e440",a,en:none,"en:to-be-completed,en:nutrition-facts-complete...",unknown,"en:fat-in-moderate-quantity,en:saturated-fat-i...",unknown,en:syrups,https://images.openfoodfacts.org/images/produc...,360.000000,1530.0,10.000000,0.000,8.0,49.000000,0.020000,2021,2024
2,3,http://world-en.openfoodfacts.org/product/0000...,Feuchtes Toilettentuch - Kamille,unknown,en:unknown,en:kein-rock-gleicht-den-anderen-wir-haben-nac...,"en:palm-oil-content-unknown,en:vegan-status-un...",en:none,en:none,0.0,en:none,unknown,en:none,"en:to-be-completed,en:nutrition-facts-complete...",unknown,en:unknown,unknown,en:none,https://images.openfoodfacts.org/images/produc...,160.000000,669.0,3.200000,0.002,1.4,8.700000,0.807087,2024,2024
3,4,http://world-en.openfoodfacts.org/product/0000...,Marinara,unknown,"en:snacks,en:sweet-snacks,en:cocoa-and-its-pro...","en:cocoa-paste,en:plant,en:cocoa,en:cane-sugar...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,0.0,en:none,e,"en:sugary-snacks,en:chocolate-products","en:to-be-completed,en:nutrition-facts-complete...",d,"en:fat-in-low-quantity,en:saturated-fat-in-hig...",unknown,en:dark-chocolate-bar,https://images.openfoodfacts.org/images/produc...,56.451613,2401.0,2.016129,-1.000,35.0,1.612903,0.000000,2019,2024
4,5,http://world-en.openfoodfacts.org/product/0000...,Bio inulin,Glass,"en:plant-based-foods-and-beverages,en:plant-ba...","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",en:none,en:none,0.0,en:none,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",unknown,en:vegetables,https://images.openfoodfacts.org/images/produc...,208.000000,840.0,0.000000,-1.000,8.0,0.000000,0.230000,2020,2024
5,7,http://world-en.openfoodfacts.org/product/0000...,Banquet mega bowls,unknown,en:butfalo-mac-and-cheese,"es:honig-stillende-frauen-nicht-geeignet,es:d-...","en:palm-oil-content-unknown,en:vegan-status-un...",en:none,en:none,0.0,en:none,unknown,en:none,"en:to-be-completed,en:nutrition-facts-to-be-co...",unknown,en:unknown,"top-75-percent-scans-2023,top-80-percent-scans...",en:butfalo-mac-and-cheese,en:none,-1.000000,-1.0,-1.000000,-1.000,-1.0,-1.000000,-1.000000,2023,2024


In [27]:
clean_data['nutriscore_grade'].value_counts()

nutriscore_grade
d                 63579
unknown           44631
c                 40347
a                 37358
e                 37233
b                 29669
not-applicable     1960
Name: count, dtype: int64